In [1]:
import pandas as pd
from thefuzz import fuzz

In [2]:
divs = pd.read_csv('section_boundaries.csv', index_col=0)

In [3]:
divs = divs[divs.start_phrase.notna()]

In [4]:
divs

,datecode,type,start_phrase,end_phrase,Unnamed: 5,Unnamed: 6
128,19940204,FOMC2,"Questions for Don? If not, let me get started....","""...to increase slightly the existing degree o...",NaN,NaN
129,19940204,FOMC1,"I want to note that revisions, if any, on the ...","Well, after some extended period of a very fla...",NaN,NaN
130,19940322,FOMC2,"Don, in the early part of your remarks, you ma...","Yes, Mr. Chairman in the sentence that was cir...",NaN,NaN
131,19940322,FOMC1,I attended my first meeting of this Committee ...,"Mr. Chairman, I think we have heard today a ve...",NaN,NaN
132,19940517,FOMC2,"Don, I just wanted to ask about the April tax ...",I'll be reading from page 15 of the Bluebook. ...,NaN,NaN
...,...,...,...,...,...,...
525,20180926,FOMC2,"Thank you, Mr. Chairman. I will be referring t...","Thank you, Mr. Chairman. The vote will be on t...",NaN,NaN
526,20181108,FOMC2,"Thank you, Mr. Chairman. I will be referring t...","Thank you, Mr. Chairman. The vote will be on t...",NaN,NaN
527,20181108,FOMC1,"Thank you, Mr. Chairman. We are pleased to pre...",Thank you. And thanks to everyone for your com...,NaN,NaN
528,20181219,FOMC2,"Thank you, Mr. Chairman. I’ll be referring to ...","Thank you, Mr. Chairman. The vote will be on t...",NaN,NaN


In [5]:
new = divs.pivot(index='datecode', columns='type', values=['start_phrase', 'end_phrase']).reset_index()
new

datecode                                       start_phrase  \
type                                                        FOMC1   
0     19940204  I want to note that revisions, if any, on the ...   
1     19940322  I attended my first meeting of this Committee ...   
2     19940517  Mike, I have a question about the inflation nu...   
3     19940706  We have tentatively indicated that next Monday...   
4     19940816  In looking at the divergence between the permi...   
..         ...                                                ...   
195   20180613  Okay. Thank you. I will be referring to the pa...   
196   20180801  Thank you, Mr. Chair. I’ll be referring to the...   
197   20180926  All those in favor? [Chorus of ayes] Thanks ve...   
198   20181108  Thank you, Mr. Chairman. We are pleased to pre...   
199   20181219  Thank you. I’ll be referring to the handout “M...   

                                                         \
type                                              FOMC2   
0     Questions for Don? If not, let me get started....   
1     Don, in the early part of your remarks, you ma...   
2     Don, I just wanted to ask about the April tax ...   
3     Questions for Don? If not, why don't I start o...   
4     Don, I interpret the recommendations you make ...   
..                                                  ...   
195   Thank you, Mr. Chairman. I would also like to ...   
196   Thank you, Mr. Chairman. I’m afraid I didn’t p...   
197   Thank you, Mr. Chairman. I will be referring t...   
198   Thank you, Mr. Chairman. I will be referring t...   
199   Thank you, Mr. Chairman. I’ll be referring to ...   

                                             end_phrase  \
type                                              FOMC1   
0     Well, after some extended period of a very fla...   
1     Mr. Chairman, I think we have heard today a ve...   
2     Mr. Chairman, what struck me most at the table...   
3     And you can do the right thing and turn out ri...   
4     Two issues are of importance to me in assessin...   
..                                                  ...   
195   Thank you, Bill. And thank you, everyone, for ...   
196   Thank you. And thanks for all your comments. I...   
197   Thank you. And thanks to everyone for really a...   
198   Thank you. And thanks to everyone for your com...   
199   Good morning, everyone. I hope everyone had a ...   

                                                         
type                                              FOMC2  
0     "...to increase slightly the existing degree o...  
1     Yes, Mr. Chairman in the sentence that was cir...  
2     I'll be reading from page 15 of the Bluebook. ...  
3     I am reading from page 26 in the Bluebook, or ...  
4     I'm reading from page 14 in the Bluebook: "In ...  
..                                                  ...  
195   Thank you. The vote will be on the monetary po...  
196   Thank you, Mr. Chairman. The vote will be on t...  
197   Thank you, Mr. Chairman. The vote will be on t...  
198   Thank you, Mr. Chairman. The vote will be on t...  
199   Thank you, Mr. Chairman. The vote will be on t...  

[200 rows x 5 columns]

In [6]:
rt = pd.read_csv('../working-csvs/raw_transcripts.csv', index_col=0)
rt = rt.dropna(subset=['content'])

In [7]:
res_dfs = []
for group in rt.groupby('date'):
    datecode = int(group[0].replace('-', '')) 
    if datecode not in divs['datecode'].unique():
        continue
    FOMC1_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC1.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC1.iloc[0]]
    FOMC2_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC2.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC2.iloc[0]]
    sections = [FOMC1_bounds, FOMC2_bounds]
    transc = group[1]
    for i, section in enumerate(sections):
        transc[f'sec{i + 1}startsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[0]) + 10], section[0]))
        transc[f'sec{i + 1}endsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[1]) + 10], section[1]))
    maxes = transc.idxmax(numeric_only=True)
    FOMC1 = transc.loc[maxes.sec1startsim:maxes.sec1endsim].copy()
    FOMC1['section'] = 1
    FOMC2 = transc.loc[maxes.sec2startsim:maxes.sec2endsim].copy()
    FOMC2['section'] = 2
    result = pd.concat([FOMC1, FOMC2])
    result = result.loc[:, ~result.columns.isin(['sec1startsim', 'sec1endsim', 'sec2startsim', 'sec2endsim'])]
    res_dfs.append(result)

In [8]:
sectioned = pd.concat(res_dfs)

In [9]:
sectioned

,speaker,content,index,date,type,section
47480,CHAIRMAN GREENSPAN.,"I want to note that revisions, if any, on the ...",72,1994-02-04,meeting,1
47481,MR. LINDSEY.,I have two questions. The first has to do with...,73,1994-02-04,meeting,1
47482,MR. PRELL.,Yes. I wouldn't want to put too fine a point o...,74,1994-02-04,meeting,1
47483,MR. LINDSEY.,Could I get a copy of that breakdown?,75,1994-02-04,meeting,1
47484,MR. PRELL.,You're trying to get inside the factory here. ...,76,1994-02-04,meeting,1
...,...,...,...,...,...,...
1270,MR. KASHKARI.,"Thank you, Mr. Chairman. If I might, I wanted...",248,2018-12-19,meeting,2
1271,CHAIRMAN POWELL.,Thank you. Vice Chair Williams.,249,2018-12-19,meeting,2
1272,VICE CHAIRMAN WILLIAMS.,"Thank you, Mr. Chairman. I support alternativ...",250,2018-12-19,meeting,2
1273,CHAIRMAN POWELL.,Thank you. And thanks to everybody. Particul...,251,2018-12-19,meeting,2


In [10]:
sectioned.to_csv('../working-csvs/raw_transcripts_sectioned.csv')